In [31]:
# LSTM RNN
import pandas as pd

In [32]:
data = pd.read_csv('Data/train.csv')
# 1 -> Fake
# 0 -> Not Fake

In [33]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [34]:
# Remove label from data

X = data.drop('label', axis=1)

In [35]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [36]:
# Store results

Y = data['label']

In [37]:
X.shape

(20800, 4)

In [38]:
Y.shape

(20800,)

In [39]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [40]:
# Vocabulary size
vocab_size = 5000

In [41]:
# Onehot Representation
messages = X.copy()

In [42]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [17]:
messages.reset_index(inplace = True)

In [18]:
# Stemming and stop word removal

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
#     type(messages['title'][i])
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [20]:
# One Hot encoding 

onehot_data = [one_hot(words, vocab_size) for words in corpus] 
onehot_data

[[3266, 2249, 1190, 4968, 4440, 2370, 3946, 1356, 200, 2186],
 [4563, 4613, 121, 3689, 924, 812, 536],
 [3131, 3655, 1427, 3930],
 [3160, 3033, 190, 3008, 2249, 220],
 [409, 924, 4189, 4685, 4305, 2026, 924, 4704, 1559, 2124],
 [325,
  2199,
  4605,
  788,
  2256,
  1821,
  2924,
  4539,
  840,
  2397,
  3678,
  2806,
  2898,
  4801,
  536],
 [429, 429, 3913, 790, 4006, 4760, 1834, 2480, 4369, 3020, 3750, 4054],
 [4884, 757, 3624, 4722, 939, 251, 3790, 1576, 1598, 1799, 2563],
 [547, 1123, 428, 3325, 1821, 1026, 4133, 1119, 3602, 2360, 1598, 1799, 2563],
 [3037, 1992, 642, 4328, 2088, 125, 1821, 2606, 1598, 1799, 2563],
 [4018, 3917, 3260, 1750, 2122, 1714, 2663, 346, 1821, 3465],
 [1590, 1313, 1585, 504, 1163, 3958, 4977, 48],
 [4359, 2826, 1391, 4547, 3549, 1243, 854, 1060, 4812, 2491, 4849],
 [3008, 4125, 4440, 1714, 1821, 2088],
 [2777, 769, 1871, 3408, 4257, 3679, 2568, 3752, 3164],
 [1341, 710, 3990, 3383, 35, 1485, 557, 1598, 1799, 2563],
 [1623, 4022, 855, 4179, 3275, 1598, 179

In [21]:
# Embedding

sentence_length = 20
embedded_data = pad_sequences(onehot_data, padding='pre', maxlen = sentence_length)
print(embedded_data)

[[   0    0    0 ... 1356  200 2186]
 [   0    0    0 ...  924  812  536]
 [   0    0    0 ... 3655 1427 3930]
 ...
 [   0    0    0 ... 1598 1799 2563]
 [   0    0    0 ... 2353  586  885]
 [   0    0    0 ...   34 2485 2059]]


In [22]:
embedded_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3266,
       2249, 1190, 4968, 4440, 2370, 3946, 1356,  200, 2186])

In [23]:
# Creating LSTM model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sentence_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_data), Y.shape

(20800, (20800,))

In [25]:
import numpy as np

X_final = np.array(embedded_data)
Y_final = np.array(Y)

In [26]:
X_final.shape, Y_final.shape

((20800, 20), (20800,))

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

In [29]:
len(X_train), len(y_train), len(X_test), len(y_test)

(13936, 13936, 6864, 6864)

In [32]:
# Training

model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 64, epochs = 10)

Epoch 1/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0068 - accuracy: 0.9979 - val_loss: 0.5015 - val_accuracy: 0.9225
Epoch 2/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0057 - accuracy: 0.9983 - val_loss: 0.4945 - val_accuracy: 0.9228
Epoch 3/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0051 - accuracy: 0.9981 - val_loss: 0.5259 - val_accuracy: 0.9145
Epoch 4/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0062 - accuracy: 0.9981 - val_loss: 0.4359 - val_accuracy: 0.9162
Epoch 5/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.6245 - val_accuracy: 0.9130
Epoch 6/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 0.6081 - val_accuracy: 0.9191
Epoch 7/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0032 - accuracy: 0.9991 - val_loss: 0.6626 - val_accuracy: 0.9219

In [31]:
tf.__version__

'2.2.0'

In [36]:
# Prediction

y_pred = model.predict(X_test)
y_pred

array([[1.0000000e+00],
       [1.0000000e+00],
       [1.3075370e-10],
       ...,
       [2.6358118e-10],
       [4.5194838e-08],
       [9.5314228e-09]], dtype=float32)

In [37]:
# import matplotlib.pyplot as plt

In [38]:
# Custom Confusion matrix plotter

# def plot_confusion_matrix(cm, classes,
#                           normalize = False,
#                           title = 'Confusion matrix',
#                           cmap = plt.cm.Blues):
    
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

In [39]:
# from sklearn import metrics

# cm = metrics.confusion_matrix(y_test, y_pred)
# plot_confusion_matrix(cm, classes = ['FAKE', 'REAL'])

In [40]:
# #Adding Dropout

# from tensorflow.keras.layers import Dropout

# embedding_vector_features = 40
# model = Sequential()
# model.add(Embedding(vocab_size, embedding_vector_features, input_length = sentence_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation = 'sigmoid'))
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [102]:
# metrics.confusion_matrix(y_test, y_pred)

In [41]:
# metrics.accuracy_score(y_test, y_pred)

In [66]:
predictions = []
for i in range(len(y_pred)):
    if y_pred[i].item() > 0.5:
#         predictions.append(str(i) + " is Fake " + str(1))
        predictions.append(1)
    else:
#         predictions.append(str(i) +" is Not Fake " + str(0))
        predictions.append(0)

In [67]:
predictions

[1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,


In [51]:
# model.evaluate(X_test, y_test)

In [69]:
results = model.evaluate(X_test, y_test, batch_size = 64)
print("test loss, test acc:", results)

108/108 [==============================] - 0s 4ms/step - loss: 0.7654 - accuracy: 0.9140
test loss, test acc: [0.7653886675834656, 0.9140442609786987]


In [68]:
# Creating classification report 
from sklearn import metrics

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3449
           1       0.92      0.91      0.91      3415

    accuracy                           0.91      6864
   macro avg       0.91      0.91      0.91      6864
weighted avg       0.91      0.91      0.91      6864

